In [1]:
# -------------------------------
# Libraries
# -------------------------------
%matplotlib inline
from time import time
import matplotlib.pylab as plt
import os.path
from math import *
import numpy as np
from tramway.helper import *
from tramway.helper.simulation import *
from tramway.helper.tessellation import *
from tramway.helper.inference import *
from tramway.plot.mesh import plot_delaunay
from numpy.random import seed
#from scipy.stats import skellam
#from scipy.optimize import fmin,minimize

#--------------------------------
# Functions
#--------------------------------
def D_field(r,t):
    '''Defining the diffusivity field: Gaussian in space, constant in time.'''
    return D - (D-D0)*np.exp(-np.sum((r/r_scale-mu_r)**2)/sig_r) 

def new_tracers(t):
    '''Number of tracers to appear in each frame n ~ Poisson(mu_on).'''
    return np.random.poisson(mu_on)

def inner_cells(cells_):
    indices = cells_.tessellation.cell_vertices
    vertices = cells_.tessellation.vertices
    cells_to_include = []
    for i in range(len(indices)):
        cells_to_include.append( not ((vertices[indices[i]]<=0)|(vertices[indices[i]]>=r_scale)).any() )
    return cells_to_include

#--------------------------------
# Constants and parameters:
#--------------------------------
# Tesselation method and name:
tessellation_method = 'hexagon'
mesh_label = '{}'.format(tessellation_method) #, location_count, min_location_count)
min_loc_count = 0
# Optimization parameters:
nt_method = ('nT_test02_hex', 'NM')
tol = 1e-2

# Normalized placement and with of diffusivity well:
mu_r=np.array([0.5,0.5]) 
sig_r=0.05
# Size of the bounding box:
r_scale = 10. # [um]
# Reference distance to set mesh size:
ref_distance = 1. # [mu]

# baseline diffusivity:
D = 0.5 # [um^2.s^-1]
# diffusivity at the bottom of the sink:
D0 = 0.05 # [um^2.s^-1]
# number of trajectories:
M = 100
# time step:
dt = 0.05 # [s]
# average trajectory lifetime (+ p_off and mu_on):
tau = 0.2
p_off = dt/tau
mu_on = p_off*M
# Simulation duration:
T = 1.0 # [s]
# position noise:
sigma = 0. # [um]
# Number of ensembles to test:
Q = 1


SyntaxError: invalid syntax (base.py, line 2258)

In [ ]:
#--------------------------------
# Simulate:
#--------------------------------
seed(1234567890)
## generate the trajectories
q=0

data_nxyt = random_walk(diffusivity=D_field, duration=T, lifetime_tau=tau, single=True, reflect=True, 
                  trajectory_mean_count=M, box=[0.,0.,r_scale,r_scale], )
data_nxyt = data_nxyt.dropna()
data_xyt = data_nxyt.drop('n',axis=1)

# tesselate for trajectories:
cells_nxyt = tessellate(data_nxyt, tessellation_method, ref_distance=ref_distance, min_location_count=0,
           strict_min_location_count=min_loc_count,
           force=True, output_label=mesh_label)
# tesselate for positions:
cells_xyt = tessellate(data_xyt, tessellation_method, ref_distance=ref_distance, min_location_count=0,
           strict_min_location_count=min_loc_count,
           force=True, output_label=mesh_label)

cells_to_include = inner_cells(cells_xyt)    
cells_xyt.tessellation.cell_label = np.array(cells_to_include)
cells_nxyt.tessellation.cell_label = np.array(cells_to_include)

## plot positions and tesselations:
cell_plot(cells_nxyt, cell_indices=True)
cell_plot(cells_xyt, cell_indices=True)
# Plot numbers of positions per cell:
map_plot((pd.DataFrame(cells_xyt.location_count, columns=['N'])),cells=cells_xyt) #,cmap='inferno') 

In [ ]:
# ------------------------------------------------
# infer in D mode without spatial regularization
# ------------------------------------------------
maps_nxyt = infer(cells_nxyt, 'D', 
             sigma=sigma, verbose=False)

print(maps_nxyt)

In [ ]:
# ------------------------------------------------
# infer using nontracking (D mode without spatial regularization only)
# ------------------------------------------------
fittime = time.time()
maps_xyt = infer(cells_xyt, nt_method[0], new_cell=Locations, dt=dt, p_off=p_off, mu_on=mu_on,
        method=nt_method[1], tol=tol)
print(time.time()-fittime, "s")

In [ ]:
centers = cells_nxyt.tessellation.cell_centers
D_true = pd.DataFrame([D_field(c_,0.) for c_ in centers],columns=['D'])
maps_true = Maps(D_true)
map_plot(maps_true, cells=cells_nxyt, title=False, cmap='inferno', clim=[0,1])
map_plot(maps_true, cells=cells_nxyt, title=False, cmap='inferno', clim=[0,1], 
         output_file='map(D_true),D=[{D0},{D}],tau={tau},dt={dt},sig_r={sig_r},box={r_scale},T={T},M={M},r_cell={ref_distance},q={q}.png')

## plot the force
map_plot(maps_nxyt, cells=cells_nxyt, title=False, cmap='inferno',clim=[0,1])
map_plot(maps_nxyt, cells=cells_nxyt, title=False, cmap='inferno',clim=[0,1], 
         output_file='map(D_nxyt),D=[{D0},{D}],tau={tau},dt={dt},sig_r={sig_r},box={r_scale},T={T},M={M},r_cell={ref_distance},q={q}.png')

# other variables are 'diffusivity' and 'potential'

map_plot(maps_xyt, cells=cells_xyt, title=False, cmap='inferno',clim=[0,1])
map_plot(maps_xyt, cells=cells_xyt, title=False, cmap='inferno',clim=[0,1], 
         output_file='map(D_xyt),D=[{D0},{D}],tau={tau},dt={dt},sig_r={sig_r},box={r_scale},T={T},M={M},r_cell={ref_distance},q={q}.png')

maps_diffs = Maps(pd.DataFrame(maps_xyt.maps['D'] - maps_nxyt.maps['diffusivity'],columns=['D']))
c_lim = max([max(maps_xyt.maps['D']),-min(maps_xyt.maps['D'])])
map_plot(maps_diffs, cells=cells_xyt, title=False, cmap='seismic', clim=[-c_lim,c_lim])
map_plot(maps_diffs, cells=cells_xyt, title=False, cmap='seismic', clim=[-c_lim,c_lim],
         output_file='map(D_xyt-D_nxyt),D=[{D0},{D}],tau={tau},dt={dt},sig_r={sig_r},box={r_scale},T={T},M={M},r_cell={ref_distance},q={q}.png')


In [ ]:
# bD_nxyt = (maps_nxyt['diffusivity']['diffusivity']-D_true['D'])/(D_true['D']*np.sqrt(2./cells_nxyt.location_count))
# bD_xyt = (maps_xyt['D']['D']-D_true['D'])/(D_true['D']*np.sqrt(2./cells_xyt.location_count))
bD_nxyt = (maps_nxyt['diffusivity']['diffusivity']-D_true['D'])/(D_true['D'])
bD_xyt = (maps_xyt['D']['D']-D_true['D'])/(D_true['D'])
plt.hist(bD_nxyt.dropna().values)
plt.hist(bD_xyt.dropna().values)

In [ ]:
analysis_tree_nxyt = Analyses(data_nxyt)
analysis_tree_nxyt.add(cells_nxyt, label='mesh')
analysis_tree_nxyt['mesh'].add(maps_nxyt, label='diffusivity')
print(analysis_tree_nxyt['mesh']['diffusivity'].data.maps)
save_rwa(f'analyses_nxyt,D=[{D0},{D}],tau={tau},dt={dt},sig_r={sig_r},box={r_scale},T={T},M={M},r_cell={ref_distance},q={q}.rwa', analysis_tree_nxyt)

#maps_xyt = pd.read_csv('Estimates-1.txt', sep='\t', index_col=0, names=['diffusivity'])
analysis_tree_xyt = Analyses(data_xyt)
analysis_tree_xyt.add(cells_xyt, label='mesh')
analysis_tree_xyt['mesh'].add(maps_xyt, label='diffusivity')
print(analysis_tree_xyt['mesh']['diffusivity'].data.maps)
save_rwa(f'analyses_xyt,D=[{D0},{D}],tau={tau},dt={dt},sig_r={sig_r},box={r_scale},T={T},M={M},r_cell={ref_distance},q={q}.rwa', analysis_tree_xyt)


In [ ]:
#--------------------------------
# Simulate:
#--------------------------------
seed(1234567890)
## generate the trajectories
Q=1
Ds_nxyt = []
bDs_nxyt = []
Ds_xyt = []
bDs_xyt = []
for q in range(Q):
    print(f"{q}/{Q}")
    nxyt = random_walk(diffusivity=D_field, duration=T, lifetime_tau=tau, single=True, reflect=True, 
                      trajectory_mean_count=M, box=[0.,0.,r_scale,r_scale], )
    nxyt = nxyt.dropna()
    # tesselate for trajectories:
    cells_nxyt = tessellate(nxyt, tessellation_method, ref_distance=ref_distance, min_location_count=0,
                strict_min_location_count=min_loc_count, force=True, output_label=mesh_label)
    # tesselate for positions:
    xyt = nxyt.drop('n',axis=1)
    cells_xyt = tessellate(xyt, tessellation_method, ref_distance=ref_distance, min_location_count=0,
                strict_min_location_count=min_loc_count, force=True, output_label=mesh_label)
    # ------------------------------------------------
    # infer in D mode without spatial regularization
    # ------------------------------------------------
    centers = cells_nxyt.tessellation.cell_centers
    D_true = pd.DataFrame([D_field(c_,0.) for c_ in centers],columns=['D'])
    
    maps_nxyt = infer(cells_nxyt, 'D', 
                 sigma=sigma, verbose=False)
    D_nxyt = maps_nxyt['diffusivity']['diffusivity']
    Ds_nxyt.append(D_nxyt)
    bD_nxyt = (D_nxyt-D_true['D'])/(D_true['D']*np.sqrt(2./cells_nxyt.location_count))
    bDs_nxyt.extend(bD_nxyt.dropna().values)
    
    maps_xyt = infer(cells_xyt, nt_method[0], new_cell=Locations, dt=dt, p_off=p_off, mu_on=mu_on,
                method=nt_method[1], tol=tol)
    D_xyt = maps_xyt['D']['D']
    Ds_xyt.append(D_xyt)
    bD_xyt = (D_xyt-D_true['D'])/(D_true['D']*np.sqrt(2./cells_xyt.location_count))
    bDs_xyt.extend(bD_xyt.dropna().values)


In [ ]:
plt.hist(bDs_nxyt)
plt.hist(bDs_xyt)

In [ ]:
help(cell_plot)

In [ ]:
xyt

In [ ]:
nxyt

In [ ]:
print(cells_xyt.points)

In [ ]:
cell_plot(cells_xyt, cell_indices=True)
